# list_unused_code_environments
This has been tested and validated on Dataiku DSS v14.1.2, running on Alma and Ubuntu Linux. It should work with the "Python 3 (ipykernel)" built-in code environment.

Written by Tim H - Sept 2025

In [0]:
import shutil
import glob
import os
import dataiku
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# Configuration

In [0]:
client = dataiku.api_client()
code_envs = client.list_code_envs()

code_env_usages = []

In [0]:
def _process_code_env(code_env_info):
    client = dataiku.api_client()
    envName = code_env_info['envName']
    code_env = client.get_code_env(code_env_info['envLang'], envName)
    
    usages = code_env.list_usages()
    print(usages)
    if usages == None:
        count = 0
    else:
        count = len(usages)
    code_env_usages.append({'CodeEnvName': envName, 'NumUsages': count})

In [0]:
max_workers = os.cpu_count() or 1
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    executor.map(_process_code_env, code_envs)
    
print('\nFinished.')

In [0]:
df = pd.DataFrame(code_env_usages).sort_values(by=["NumUsages", "CodeEnvName"], ascending=False)

In [0]:
df